In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings


vectorstore = FAISS.from_texts(
    ["bharath focuses on providing content on Data Science, AI, ML, DL, CV, NLP, Python programming, etc. in English."], embedding = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite-preview-02-05")

retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("What is bharath?")

'bharath provides content on Data Science, AI, ML, DL, CV, NLP, Python programming, etc. in English.'

In [3]:
retrieval_chain.invoke("bharath provides in what language?")

'bharath provides in English.'

In [7]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import GoogleGenerativeAI
llm1 = GoogleGenerativeAI(model="gemini-2.0-pro-exp-02-05")

vectorstore = FAISS.from_texts(
    ["bharath focuses on providing content on Data Science, AI, ML, DL, CV, NLP, Python programming, etc. in English."], embedding = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "Who is bharath?", "language": "hindi"})

'भरत कौन है, यह जानकारी उपलब्ध नहीं है।'